In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display
from colavsim import colav_similarity, parse_doi, parse_string

In [11]:
es = Elasticsearch(hosts="172.19.31.8")

In [13]:
openalex = list(MongoClient()["openalexco"]["works"].find({'doi': {"$ne": None}}))

In [14]:
len(openalex)

203863

In [15]:
es_index = "openalex_raw"
recreate = True
if recreate:
    if es.indices.exists(index=es_index):
        es.indices.delete(index=es_index)
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index)
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        work = {}
        work["title"] = i["title"]
        work["authors"] = ""
        for j in i["authorships"]:
            if "display_name" in j["author"].keys(): 
                if j["author"]["display_name"] is not None:
                    work["authors"] += " " + j["author"]["display_name"]
        work["source"] = i["host_venue"]["display_name"]
        work["year"] = i["publication_year"]
        work["volume"] = i["biblio"]["volume"] 
        work["issue"] = i["biblio"]["issue"] 
        work["first_page"] = i["biblio"]["first_page"]
        work["last_page"] = i["biblio"]["last_page"]
        work["doi"] = i["doi"] 
    
        entry = {"_index": es_index,
                 "_id": str(i["_id"]),
                 "_source": work}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

In [8]:
# settings = {
#   "settings": {
#     "index": {
#       "similarity": {
#         "default": {
#           "type": "DFR",
#           "basic_model": "g",
#           "after_effect": "b",
#           "normalization": "h2",
#           "normalization.h2.c": "600.0"  # Cambia el valor según tus necesidades
#         }
#       }
#     }
#   }
# }
# settings = {
#   "settings": {
#     "index": {
#       "similarity": {
#         "default": {
#           "type": "BM25",
#           "b": 0.80,   # Ajusta el parámetro "b" de BM25
#           "k1": 2.1   # Cambia el valor según tus necesidades
#         }
#       }
#     }
#   }
# }
settings={}

In [16]:
for i in range(len(openalex)):
    openalex[i]['title'] = parse_string(openalex[i]['title'])

In [17]:
es_index = "openalex_parsed"
recreate = True
if recreate:
    if es.indices.exists(index=es_index):
        es.indices.delete(index=es_index)
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index)

In [18]:
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        work = {}
        work["title"] = i["title"]
        work["authors"] = ""
        for j in i["authorships"]:
            if "display_name" in j["author"].keys(): 
                if j["author"]["display_name"] is not None:
                    work["authors"] += " " + j["author"]["display_name"]
        work["source"] = i["host_venue"]["display_name"]
        work["year"] = i["publication_year"]
        work["volume"] = i["biblio"]["volume"] 
        work["issue"] = i["biblio"]["issue"] 
        work["first_page"] = i["biblio"]["first_page"]
        work["last_page"] = i["biblio"]["last_page"]
        work["doi"] = i["doi"] 
    
        entry = {"_index": es_index,
                 "_id": str(i["_id"]),
                 "_source": work}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

In [12]:
settings = {
  "settings": {
    "index": {
      "similarity": {
        "default": {
          "type": "DFR",
          "basic_model": "g",
          "after_effect": "b",
          "normalization": "h1",
          # "normalization.h2.c": "600.0"  # Cambia el valor según tus necesidades
        }
      }
    }
  }
}



In [13]:
es_index = "openalex_dfr"
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index,body=settings)

/tmp/ipykernel_39320/4260525492.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=es_index,body=settings)


In [14]:
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        entry = {"_index": es_index,
                 "_id": counter,
                 "_source": i}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1

In [ ]:
settings = {
  "settings": {
    "index": {
      "similarity": {
        "default": {
          "type": "LMDirichlet"
        }
      }
    }
  }
}

In [15]:
es_index = "openalex_lmdirichlet"
if not es.indices.exists(index=es_index):
    es.indices.create(index=es_index,body=settings)

/tmp/ipykernel_39320/2560368478.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=es_index,body=settings)


In [16]:
if recreate:
    request_timeout = 60
    bulk_size = 100

    es_entries = []
    counter = 0
    for i in openalex:
        entry = {"_index": es_index,
                 "_id": counter,
                 "_source": i}
        es_entries.append(entry)
        if len(es_entries) == bulk_size:
            helpers.bulk(es, es_entries, refresh=True,request_timeout=request_timeout)
            es_entries = []
        counter +=1